<a href="https://colab.research.google.com/github/Jose-Augusto-C-M/KettleHole_SLIC_CNN/blob/main/main_segcc_proba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model, load_model
from numpy import resize, expand_dims
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.xception import decode_predictions
from keras import backend as K
import glob
from skimage import segmentation
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float, img_as_ubyte
import tensorflow as tf
from skimage import io
import numpy as np
from skimage.util import img_as_float, img_as_ubyte
from PIL import Image
from numpy import resize, expand_dims
from IPython.display import HTML, display
from keras.preprocessing.image import ImageDataGenerator
import pylab
from tqdm import tqdm
import time
import csv
import os
import functools
import multiprocessing
from multiprocessing import Process, Manager
import threading
import tensorflow as tf
import matplotlib.image as mpimg
import scipy

In [ ]:
def synchronized(wrapped):
    lock = threading.Lock()
    @functools.wraps(wrapped)
    def _wrap(*args, **kwargs):
        with lock:
            return wrapped(*args, **kwargs)
    return _wrap
class Classfield_segmented(object):
       c = csv.writer(open("dados.csv", "w", newline=''))
       location_drive='/content/drive/MyDrive/datasets/dataset_anette/data_labelme/' #caminho até onde está a base das pastas
       IMG_WIDTH, IMG_HEIGHT = 224,224
       dict_classes=['background','Cirsium_arvense','Salix spec','Oenanthe aquatica','Phalaris arundinacea','Urtica dioica','Arctium spec.','Carex riparia live','Salix cinerea','Typha live', 'Phragmites australis live','Phragmites australis mix live and dead','Nymphaeidae','water']
        #'01':'background', '02':'Cirsium arvense', '03':'Salix spec', '04':'Oenanthe aquatica', '05': 'Phalaris arundinacea', '06': 'Urtica dioica', '07': 'Arctium spec.', '08': 'Carex riparia live','09': 'Salix cinerea','10': 'Typha live','11': 'Phragmites australis live','12': 'Phragmites australis mix live and dead','13':'Nymphaeidae','14': 'water'
        #background,'Cirsium_arvense','Salix spec','Oenanthe aquatica','Phalaris arundinacea','Urtica dioica','Arctium spec.','Carex riparia live','Salix cinerea','Typha live', 'Phragmites australis live','Phragmites australis mix live and dead','Nymphaeidae','water'
       color_classes=[(0, 0, 0),(255,192,203),(200, 200, 74 ),(200, 190, 167),(200, 137, 118),(62, 200, 41),(200, 37, 168),(146, 200, 7),(200, 179, 37),(165, 200, 48),(200, 148, 179),(200, 167, 193),(200, 107, 163),(122, 144, 200 )]
       model=None  
       
       def getRAMinfo(self):
         p = os.popen('free')
         i = 0
         while 1:
            i = i + 1
            line = p.readline()
            if i==2:
              return(line.split()[1:4])
              
       @synchronized
       
       def predict(self,image):
          predict=self.model.predict(image)
        #   image_path = self.
          print(predict)
          with open('Refs.txt', 'a',encoding='utf-8') as f:
            for line in predict:
                f.write(str(line))
                f.write('\n')
            
            

        #   return scipy.special.softmax(predict, axis=1) 

          return np.argmax(predict, axis=1)

    #    print('image: ', image_path)

       def job_process(self,model_name,filename, preprocess_input, decode_predictions,list_iou):
            print(filename)
            im=io.imread(filename)
            #plt.imshow(im)
            
            #plt.show()
            
            #Create SLIC

            img_load=im.copy()
            ##create a superpixels
            segments = segmentation.slic(img_load, n_segments=4000,compactness=10, sigma=5)
            # segments = img_load
            #mark boundaries
            img_slic=mark_boundaries(img_load, segments)
            
            mpimg.imsave(self.location_drive + "result/"+model_name+"/slic/"+filename.split("image/")[1],img_slic) #local do drive

            #Classificando os superpixels
            #print(segments)
            #superpixels=[]
            pred_list=[]
            qtd=len(np.unique(segments))
            print("Processando os superpixels da imagem")
            img_paint=im.copy()    
            for (i, seg) in tqdm(enumerate(np.unique(segments )), desc = 'tqdm() Progress Bar Segments'):
              # Create a mask, painting black all pixels outside of segment and white the pixels inside.
              mask_segment = np.zeros(  img_load.copy().shape[:2], dtype="uint8")
              mask_segment[segments == seg] = 255

              size_segment = mask_segment[segments == seg].size
              segment =  img_load.copy() 
              segment = cv2.bitwise_and(segment, segment, mask=mask_segment)
              # Get the countours around the segment
              contours, _  = cv2.findContours(mask_segment,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]
         
              m = -1
              max_contour = None
              for cnt in contours:
                       if (len(cnt) > m):
                          m = len(cnt)
                          max_contour = cnt

              # Get the rectangle that encompasses the countour
              x,y,w,h = cv2.boundingRect(max_contour)
              segment = segment[y:y+h, x:x+w]

              #superpixels.append(segment)
         
              # pre-process the image for classification
              image = cv2.resize(segment, (self.IMG_HEIGHT, self.IMG_WIDTH))
              image = image.astype("float") / 255.0
              image = img_to_array(image)
              image = np.expand_dims(image, axis=0)
           
              predict = self.predict(image)
              
              color=None

                 
              for cont, name_classs in enumerate(self.dict_classes):  
                if(self.dict_classes[predict[0]]==name_classs):
                    color=self.color_classes[cont]
                    break
              #predict = np.argmax(predict, axis=1)
              #print(predict)
              pred_list.append(self.dict_classes[predict[0]])
              #if(idx_segment<4):
                #plt.imshow(segment)
                #print(dict_classes[predict[0]])
                #plt.show()
                #break
              
              #painting  
              height, width, channels = img_paint.shape   
               
              mask_inv = cv2.bitwise_not(mask_segment)
               
              # Paint all pixels in original image with choosed color
              class_color = np.zeros((height,width,3), np.uint8)
              class_color[:, :] = color
              colored_image = cv2.addWeighted(img_paint, 0.0, class_color, 1.0, 0)
               
              colored_image = cv2.bitwise_and(colored_image, colored_image, mask=mask_segment)
              clear = False
              # Create a new image keeping the painting only in pixels inside of segments
              new_image =   img_paint
              new_image = cv2.bitwise_and(new_image, new_image, mask=mask_inv)
              mask_segment[:] = 255
              img_paint = cv2.bitwise_or(new_image, colored_image, mask=mask_segment)
               
             

            #/content/drive/MyDrive/datasets/dataset_anette/data_labelme/
            mpimg.imsave(self.location_drive + "result/"+model_name+"/paint/"+filename.split("image/")[1],img_paint)
            ##Calcula o IoU
            
            """
            im_mask=cv2.imread(self.location_drive+"sheep_mask/"+filename.split("sheep/")[1].split("_img")[0]+"_label.png")
            print(self.location_drive+"sheep_mask/"+filename.split("sheep/")[1].split("_img")[0]+"_label.png")
            #print(im_mask)
            im_mask = cv2.cvtColor(im_mask, cv2.COLOR_BGR2GRAY)
            ret,im_mask = cv2.threshold(im_mask,25, 255, cv2.THRESH_BINARY)
            #im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
            #plt.imshow(im_mask ,cmap=pylab.gray())
            ##target, predition
            img_p=cv2.cvtColor(img_paint, cv2.COLOR_BGR2GRAY)
            r,img_p = cv2.threshold(img_p, 25, 255, cv2.THRESH_BINARY)
            intersection = np.logical_and(np.array(im_mask), np.array(img_p))
            union = np.logical_or(im_mask,img_p)
            
            mpimg.imsave("result/"+model_name+"/inter/"+filename.split("sheep/")[1],intersection)
            
            mpimg.imsave("result/"+model_name+"/union/"+filename.split("sheep/")[1],union)
            iou_score = np.sum(intersection) / np.sum(union)
            print(iou_score)
            list_iou.append(iou_score)
            
            self.c.writerow([str(model_name),str(filename.split("sheep/")[1]),str(iou_score)])             
            """
     
       def process(self):       
              dict_preprocessing = {}
              dict_preprocessing[1] = applications.vgg16.preprocess_input, applications.vgg16.decode_predictions
              dict_preprocessing[2] = applications.inception_v3.preprocess_input, applications.inception_v3.decode_predictions
              dict_preprocessing[3] = applications.densenet.preprocess_input, applications.densenet.decode_predictions
              dict_preprocessing[4] = applications.resnet_v2.preprocess_input, applications.resnet_v2.decode_predictions
              dict_preprocessing[5] = applications.xception.preprocess_input, applications.xception.decode_predictions

              self.c.writerow(["Modelo","Arquivo","IOU"])
              #names of models
              list_model=["Xception"]#,"ResNet152V2" , "InceptionV3","VGG16", "DenseNet201"]
              for model_name in tqdm(list_model, desc = 'tqdm() Progress Bar Models'):
                K.clear_session()
                self.model = load_model(self.location_drive+"model/"+model_name+"_transfer_learning_adagrad.h5")
                preprocess_input=None
                decode_predictions=None
                if model_name=="VGG16":
                   preprocess_input, decode_predictions = dict_preprocessing[1]
                if model_name=="InceptionV3":
                   preprocess_input, decode_predictions = dict_preprocessing[2]
                if model_name=="DenseNet201":
                   preprocess_input, decode_predictions = dict_preprocessing[3]
                if model_name=="ResNet152V2":
                   preprocess_input, decode_predictions = dict_preprocessing[4]
                if model_name=="Xception":
                   preprocess_input, decode_predictions = dict_preprocessing[5]   
                list_iou=[]
                print("Load Model H5:"+model_name+"_transfer_learning_adagrad.h5")
                # print(decode_predictions(prediction, top=3)[0])
                #Process each image
                threads=[]
                for filename in tqdm(glob.iglob(self.location_drive+'image/*.png', recursive=True), desc = 'tqdm() Progress Bar Files'):
                   self.job_process(model_name,filename,preprocess_input, decode_predictions,list_iou)
                   #th = threading.Thread(target=self.job_process,args=(model_name,filename,preprocess_input, decode_predictions,list_iou ))
                   #threads.append(th)
                
                # Output is in kb, here I convert it in Mb for readability
                RAM_stats = self.getRAMinfo()
                RAM_total = round(int(RAM_stats[0]) / 1000,1)
                RAM_used = round(int(RAM_stats[1]) / 1000,1)
                print("RAM Total : "+str(RAM_total))
                print("RAM Used : "+str(RAM_used))
                 
                print("Wait a moment, the threads are processing "+str(len(threads)) +" images, it may be delayed depending on the size or quantity of the images!")
                with tqdm(total=len(threads)) as pbar:
                  for  t in threads:
                        t.start()
                        #t.join()
                        if((RAM_total)<33000):#se menor que 10gb
                             RAM_stats = self.getRAMinfo()
                             RAM_used = round(int(RAM_stats[1]) / 1000,1)
                             if((RAM_total-RAM_used)<6000):
                               t.join()
                             pbar.update(1)
                  pbar.close()
                for  t in threads:
                   t.join()
                #print("Average IoU"+ str(np.mean(list_iou)))
                #print("Model_name:"+list_iou)

if __name__ == "__main__":
    process_file= Classfield_segmented()
    process_file.process()




tqdm() Progress Bar Models:   0%|          | 0/1 [00:00<?, ?it/s]

Load Model H5:Xception_transfer_learning_adagrad.h5



tqdm() Progress Bar Files: 0it [00:00, ?it/s]

/content/drive/MyDrive/datasets/dataset_anette/data_labelme/image/100_0002_0033.png


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


...

[[7.67434482e-03 1.90106466e-01 2.03089286e-02 1.13518254e-04
  7.56530106e-01 1.49071617e-02 1.95717992e-04 4.28449176e-03
  1.58456445e-04 1.49010026e-04 5.57182683e-03]]




tqdm() Progress Bar Segments: 1507it [13:15,  1.95it/s]

[[4.5740453e-06 1.6584456e-03 1.2170020e-07 2.9581741e-05 4.3796470e-05
  2.3863788e-05 2.2271703e-07 9.9819916e-01 2.6265877e-06 9.7318900e-07
  3.6597648e-05]]




tqdm() Progress Bar Segments: 1508it [13:15,  1.90it/s]
tqdm() Progress Bar Files: 0it [14:11, ?it/s]
tqdm() Progress Bar Models:   0%|          | 0/1 [14:15<?, ?it/s]


KeyboardInterrupt: ignored